<a href="https://colab.research.google.com/github/suhyeon0325/TIL/blob/main/finance_news_in_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 금융 뉴스를 감성 분류하는 AI 모델 만들기
- GPT-3의 파인튜닝 기능 이용하여 금융 뉴스 문장에 대해 긍정, 부정, 중립을 분류하는 AI 모델 만들기

In [ ]:
!pip install gradio pandas openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 64.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 13.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.9/307.9 kB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 52.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 9.0 MB

## 구글 드라이브 연동

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import openai
import gradio as gr
import pandas as pd
from openai import OpenAI

- OPENAI Key 입력

In [ ]:
key = 'Open-ai-key'
openai.api_key = key

## 데이터셋 준비하기
- Finance Phrase Bank 데이터셋
    + 참조 : https://github.com/ukairia777/finance_sentiment_corpus

In [ ]:
import pandas as pd
DATA_PATH = '/content/drive/MyDrive/소스코드/data/'
data = pd.read_csv(DATA_PATH + "finance_data.csv")
data = data.drop_duplicates().reset_index(drop=True)
data.head()

,labels,sentence,kor_sentence
0,neutral,"According to Gran, the company has no plans to...","Gran에 따르면, 그 회사는 회사가 성장하고 있는 곳이지만, 모든 생산을 러시아로..."
1,neutral,Technopolis plans to develop in stages an area...,테크노폴리스는 컴퓨터 기술과 통신 분야에서 일하는 회사들을 유치하기 위해 10만 평...
2,negative,The international electronic industry company ...,"국제 전자산업 회사인 엘코텍은 탈린 공장에서 수십 명의 직원을 해고했으며, 이전의 ..."
3,positive,With the new production plant the company woul...,새로운 생산공장으로 인해 회사는 예상되는 수요 증가를 충족시킬 수 있는 능력을 증가...
4,positive,According to the company's updated strategy fo...,"2009-2012년 회사의 업데이트된 전략에 따르면, Basware는 20% - 4..."


- labels 열은 neural, negative, positive로 구성됨.
- sentence는 영어로 기재되어 있고, kor_sentence는 한국어로 작성되어 있음

## 데이터 전처리하기
- 데이터 프레임으로 prompt와 completion 형태로 변환 처리한 후, json으로 저장한다.

In [ ]:
data['prompt'] = data['kor_sentence']
data['completion'] = data['labels']
data = data[['prompt', 'completion']]

data.head()

,prompt,completion
0,"Gran에 따르면, 그 회사는 회사가 성장하고 있는 곳이지만, 모든 생산을 러시아로...",neutral
1,테크노폴리스는 컴퓨터 기술과 통신 분야에서 일하는 회사들을 유치하기 위해 10만 평...,neutral
2,"국제 전자산업 회사인 엘코텍은 탈린 공장에서 수십 명의 직원을 해고했으며, 이전의 ...",negative
3,새로운 생산공장으로 인해 회사는 예상되는 수요 증가를 충족시킬 수 있는 능력을 증가...,positive
4,"2009-2012년 회사의 업데이트된 전략에 따르면, Basware는 20% - 4...",positive


In [ ]:
data.to_json(DATA_PATH + 'finance_data.jsonl', orient='records', force_ascii=False, lines=True)

## 데이터 준비 도구
- OpenAI에서 제공하는 데이터 준비 도구 활용

In [ ]:
!openai tools fine_tunes.prepare_data -f "/content/drive/MyDrive/소스코드/data/finance_data.jsonl"

Analyzing...

- Your file contains 4840 prompt-completion pairs
- Based on your data it seems like you're trying to fine-tune a model for classification
- For classification, we recommend you try one of the faster and cheaper models, such as `ada`
- For classification, you can estimate the expected model performance by keeping a held out dataset, which is not used for training
- There are 10 duplicated prompt-completion sets. These are rows: [1393, 2885, 2886, 2888, 3046, 3047, 3344, 3532, 3611, 3932]
- More than a third of your `prompt` column/key is uppercase. Uppercase prompts tends to perform worse than a mixture of case encountered in normal language. We recommend to lower case the data if that makes sense in your domain. See https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset for more details
- Your data does not contain a common separator at the end of your prompts. Having a separator string appended to the end of the prompt makes it clearer to the fine-tu

## 데이터 파인 튜닝 도구
- 파일 업로드 후, 파인 튜닝 진행하는 단계로 넘어간다.
- 정상적으로 실행된 경우 status의 값이 uploaded로 나오며, id값이 발급됨.

In [ ]:
from openai import OpenAI
# import os
# os.environ["OPENAI_API_KEY"] = key
# import dotenv
# env_file = dotenv.find_dotenv()
# dotenv.load_dotenv(env_file)
api_key = "open-ai-key"

#  모델링 학습 코드, 데이터 준
client = OpenAI(api_key=api_key)
client.files.create(
    file = open(DATA_PATH + "finance_data_prepared_train.jsonl", "rb"),
    purpose = "fine-tune"
)

FileObject(id='file-3IlROLl3ioAPBOrV6ys80Uv8', bytes=679022, created_at=1709525774, filename='finance_data_prepared_train.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

## 파인 튜닝
- 파인 튜닝을 진행하기 위해 다음 코드를 실행한다.
- FileObject의 id값을 입력한다.
- 실행결과로 학습 id값이 발급된다.  

In [ ]:
client.fine_tuning.jobs.create(
    training_file = "file-3IlROLl3ioAPBOrV6ys80Uv8",
    model = "babbage-002"
)

FineTuningJob(id='ftjob-59kmtOPQHHYeUshzCsjQoIPD', created_at=1709526230, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='babbage-002', object='fine_tuning.job', organization_id='org-JDZ21IvQVWWIOMiBRGu96oU6', result_files=[], status='validating_files', trained_tokens=None, training_file='file-3IlROLl3ioAPBOrV6ys80Uv8', validation_file=None, user_provided_suffix=None)

### 학습 현황 확인하기
- status값은 현재 학습 현황을 나타낸다.
- validating_files라고 나온다면 아직 학습이 시작되지 않음
- 일반적으로 평균 10분 내외로 학습 시작

In [ ]:
client.fine_tuning.jobs.retrieve("ftjob-59kmtOPQHHYeUshzCsjQoIPD")

FineTuningJob(id='ftjob-59kmtOPQHHYeUshzCsjQoIPD', created_at=1709526230, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size=7, learning_rate_multiplier=2), model='babbage-002', object='fine_tuning.job', organization_id='org-JDZ21IvQVWWIOMiBRGu96oU6', result_files=[], status='validating_files', trained_tokens=None, training_file='file-3IlROLl3ioAPBOrV6ys80Uv8', validation_file=None, user_provided_suffix=None)

- 학습 종료 메일이 온 후 학습 현황 코드를 다시 실행한 결과 확인
- 이 때, status가 running에서 succeeded로 변경됨을 확인한다.

In [ ]:
client.fine_tuning.jobs.retrieve("ftjob-59kmtOPQHHYeUshzCsjQoIPD")

FineTuningJob(id='ftjob-59kmtOPQHHYeUshzCsjQoIPD', created_at=1709526230, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model='ft:babbage-002:personal::8yuJ4Lzn', finished_at=1709526813, hyperparameters=Hyperparameters(n_epochs=3, batch_size=7, learning_rate_multiplier=2), model='babbage-002', object='fine_tuning.job', organization_id='org-JDZ21IvQVWWIOMiBRGu96oU6', result_files=['file-Sv4sNFdVlOCBhC6kv4z7XBG1'], status='succeeded', trained_tokens=698358, training_file='file-3IlROLl3ioAPBOrV6ys80Uv8', validation_file=None, user_provided_suffix=None)

## 전처리 마친 데이터 가져오기

In [ ]:
test = pd.read_json(DATA_PATH + "finance_data_prepared_valid.jsonl", lines = True)
test.head()

,prompt,completion
0,새로운 생산공장으로 인해 회사는 예상되는 수요 증가를 충족시킬 수 있는 능력을 증가...,positive
1,"2009-2012년 회사의 업데이트된 전략에 따르면, basware는 20% - 4...",positive
2,aspocomp의 성장기에 대한 자금 조달은 기술적으로 더 까다로운 hdi 인쇄 회...,positive
3,영업이익은 총 21.1 유로로 2007년 18.6 mn에서 증가하여 순매출의 9.7...,positive
4,텔리아소네라 tlsn은 이번 제안이 핵심 사업 보유에 대한 지분을 늘리기 위한 전략...,positive


## 모델 호출하기
- 학습된 모델을 가져옵니다.

In [ ]:
test['prompt'][0]

'새로운 생산공장으로 인해 회사는 예상되는 수요 증가를 충족시킬 수 있는 능력을 증가시키고 원자재 사용을 개선하여 생산 수익성을 높일 것이다. ->'

In [ ]:
ft_model = "ft:babbage-002:personal::8yuJ4Lzn"
res = client.completions.create(model=ft_model, prompt=test['prompt'][0], max_tokens = 1, temperature=0)
res.choices[0].text

' positive'

## 사용자 정의 함수 만들기
- input_text가 들어오면 감성 분석 결과를 반환하는 함수인 get_result() 구현

In [ ]:
def get_result(input_text):
    input_text = input_text + ' ->'
    ft_model = "ft:babbage-002:personal::8yuJ4Lzn"
    res = client.completions.create(model=ft_model, prompt=input_text, max_tokens = 1, temperature=0)
    return res.choices[0].text.strip()

In [ ]:
test = "순매출이 45% 감소함에 따라서 주가도 지속적으로 하락하고 있다."
get_result(test)

'negative'

## 금융 뉴스 감성 분류하는 모델의 UI 만들기

In [ ]:
iface = gr.Interface(fn = get_result,
                     inputs=gr.Textbox(lines=5, placeholder='감성 분석할 뉴스를 입력해주세요.'),
                     outputs = 'text',
                     title = '금융 뉴스 감성 분석',
                     description = '금융 뉴스 감성 분석')

iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://159e30703f4748e2db.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
